<a href="https://colab.research.google.com/github/immprasetyo/ibm-data-science/blob/main/plotly-dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Launch Sites Locations Analysis with Plotly

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations

In [ ]:
#python3 -m pip install pandas dash
#wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
#wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py'

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

In [ ]:
# Create an app layout
app.layout = html.Div(
    children = [
        html.H1(
            'SpaceX Launch Records Dashboard',
            style = {
                'textAlign': 'center',
                'color': 'darkseagreen',
                'font-size': 40
            }
        ),
        
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites
        dcc.Dropdown(
            id = 'site-dropdown',
            options = [
                {'label': 'ALL SITES', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'VAFB SLC-4E	', 'value': 'VAFB SLC-4E'}
            ],
            value = 'ALL',
            placeholder = 'Select a Launch Site here',
            searchable = True
        ),
        html.Br(),
        
        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(dcc.Graph(id = 'success-pie-chart')),
        html.Br(),
        html.P('Payload range (Kg):'),
        
        # TASK 3: Add a slider to select payload range
        dcc.RangeSlider(
            id = 'payload-slider',
            min = 0,
            max = 10000,
            step = 1000,
            marks = {
                0: '0',
                2000: '2000',
                4000: '4000',
                6000: '6000',
                8000: '8000',
                10000: '10000'
            },
            value = [min_payload, max_payload]
        ),
        
        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(dcc.Graph(id = 'success-payload-scatter-plot'))
    ]
)

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'), [
        Input(component_id = 'site-dropdown', component_property = 'value')
    ]
)

In [ ]:
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        ax = px.pie(
            spacex_df,
            values = 'Landing Outcome',
            names = 'Launch Site',
            title = 'Total Success Outcomes by Site'
        )
        return ax
    
    else:
        df1 = spacex_df[spacex_df['Launch Site'] == entered_site]
        df1 = df1.groupby('Landing Outcome').size().reset_index(name = 'Class Count')
        ax = px.pie(
            df1,
            values = 'Class Count',
            names = 'Landing Outcome',
            title = 'Total Success Outcomes by %s' % entered_site
        )
        return ax

In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id = 'success-payload-scatter-plot', component_property = 'figure'), [
        Input(component_id = 'site-dropdown', component_property = 'value'),
        Input(component_id = 'payload-slider', component_property = 'value')
    ]
)

In [ ]:
def get_scatter_plot(entered_site, slider_range):
    low, high = slider_range
    if entered_site == 'ALL':
        mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
        ax = px.scatter(
            spacex_df[mask],
            x = 'Payload Mass (kg)',
            y = 'Landing Outcome',
            color = 'Booster Version Category',
            title = 'Correlation between Payload and Success for All Sites'
        )
        return ax
    
    else:
        df2 = spacex_df[spacex_df['Launch Site'] == entered_site]
        mask = (df2['Payload Mass (kg)'] > low) & (df2['Payload Mass (kg)'] < high)
        ax = px.scatter(
            df2[mask],
            x = 'Payload Mass (kg)',
            y = 'Landing Outcome',
            color = 'Booster Version Category',
            title = 'Correlation between Payload and Success for %s' % entered_site
        )
        return ax

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

In [ ]:
#python3 spacex_dash_app.py